In [ ]:
import os

%pwd

# we need to jump back to the root folder 

: 

In [2]:
os.chdir('../')

%pwd

'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\Deep learning\\CNN\\End to end Chicken disease\\Chicken-Disease_classification'

In [11]:
#3. Update entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [12]:
#4. Update the configuration manager in src config
from cnnClassifier.constants import * # this will read from constant
from cnnClassifier.utils.common import read_yaml, create_directories # this will read from utlis

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
#5. update the components
import os
import urllib.request as request
import zipfile
from cnnClassifier.logging import  logger
from cnnClassifier.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
# 5. update the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-05 13:34:59,086: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-05 13:34:59,089: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-05 13:34:59,092: INFO: common: created directory at: artifacts]
[2024-05-05 13:34:59,094: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-05 13:35:07,563: INFO: 3487473262: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A7F6:368622:234638:2BB4AD:66373DAD
Accept-Ranges: bytes
Date: Sun, 05 May 2024 08:05:03 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4741-BOM
X-Cache: M